In [28]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

import numpy as np

In [2]:
import sys
path_module = r'C:\Users\mathe\OneDrive\Graduação - UFC\Engenharia da Computação\TCC\Códigos e implementações\V.2\kinematics-robotics\src\kinematicsrobotics'
sys.path.append(path_module)
import datahandler

## Dados

In [4]:
path_data = r'src\data\ready\dataset-radius-0.5cm.csv'
path_project = r'C:\Users\mathe\OneDrive\Graduação - UFC\Engenharia da Computação\TCC\Códigos e implementações\V.2\kinematics-robotics'

ext = datahandler.extract(path_project)

dataset = ext.dataframe(path_data)


### Atributos e saídas

In [5]:
y = dataset.iloc[:,0:4]
x = dataset.iloc[:,5:11]

#### Análise dos dados

In [6]:
print(dataset.columns)

Index(['theta_1', 'theta_2', 'theta_3', 'theta_4', 'theta_5', 'p_x', 'p_y',
       'p_z', 'roll', 'pich', 'yaw'],
      dtype='object')


In [7]:
print(f"Tamanho do dataset: {y.shape}")

Tamanho do dataset: (15324, 4)


In [8]:
y.head(2)

,theta_1,theta_2,theta_3,theta_4
0,0,0,0,0
1,0,0,12,0


In [9]:
x.head(2)

,p_x,p_y,p_z,roll,pich,yaw
0,54.000000,-1.102182e-15,10.000000,2.356194,-1.570796,7.853982e-01
1,53.213314,-2.477121e-15,2.515179,3.141593,-1.361357,2.752040e-16


## Pré-processamento de dados

In [10]:
import modelmlp

In [11]:
x_train, x_test, y_train, y_test = modelmlp.split(x,y,0.1)

#### informações

In [12]:
print(f"{len(x_train)}")
print(f"{len(x_test)}")

13791
1533


### Normalização dos dados

In [116]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
y_train = scaler.fit_transform(y_train)

## Modelo

In [18]:
from sklearn.model_selection import ShuffleSplit
ss = ShuffleSplit(n_splits=3, test_size=0.2, random_state=42)
for train_index, test_index in ss.split(x_train, y_train):
    print("Índices de Treino:", len(train_index))
    print("Índices de Teste:", len(test_index))
    print("----------------------")

Índices de Treino: 11032
Índices de Teste: 2759
----------------------
Índices de Treino: 11032
Índices de Teste: 2759
----------------------
Índices de Treino: 11032
Índices de Teste: 2759
----------------------


In [ ]:
from sklearn.neural_network import MLPRegressor
def modelo(hidden):
    func_act = 'tanh'
    model = MLPRegressor(hidden_layer_sizes=hidden,
                        activation=func_act)

In [126]:
# Definir os parâmetros
taxa_aprendizado = 0.01
numero_epocas = 1000
erro_adm = 0.01
fator_momento = 0.9
validacao = 0.1

In [129]:
model = MLPRegressor(hidden_layer_sizes=(100, 200), #validação
                    activation='tanh',  # Função de ativação da camada oculta
                    random_state=42)

## Seleção de modelos

In [24]:
def grid_search(X_train, y_train, param_grid, model):
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3,scoring='neg_mean_squared_error', return_train_score=True)
    grid_search.fit(X_train, y_train)
    #.ravel()

    # Avaliação do melhor hiperparâmetro
    melhor_modelo = grid_search.best_estimator_
    y_pred = melhor_modelo.predict(X_train)
    mse = mean_squared_error(y_train, y_pred)
    r2 = r2_score(y_train, y_pred)

    print("Melhores hiperparâmetros:", grid_search.best_params_)
    print(f"\nMSE nos dados completos: {mse}")
    print(f"R² nos dados completos: {r2}")

    # Extrair os resultados da validação cruzada
    resultados = pd.DataFrame(grid_search.cv_results_)

    colunas_interesse = ['param_hidden_layer_sizes', 'mean_test_score', 'std_test_score']
    resultados_interesse = resultados[colunas_interesse]

    print("Avaliação do MSE para cada hiperparâmetro testado:")
    print(resultados_interesse)
    return resultados

### Modelo

In [130]:
# Treinando o modelo
model.fit(Xtrain, Ytrain)

# Fazendo previsões
y_pred = model.predict(Xtrain)

# Avaliando o modelo
mse = mean_squared_error(Ytrain, y_pred)
print(mse)
#print(f"{Ytrain}  {y_pred}")

0.008671714073118121


## Avaliação de modelos


Métricas de regressão (MSE, RMSE, R²)
Validação cruzada
Curvas de aprendizado
Matrizes de confusão.
Grid search e busca aleatória de hiperparâmetros.

In [ ]:
from sklearn.metrics import mean_squared_error

def predict_mse(model,X_test,Y_test):
    y_predic = model.predict(X_test)
    return mean_squared_error(Y_test, y_predic,multioutput='raw_values')